## Visualise fitting results 

In [ ]:
include("main_utils.jl")

default_plot_setting()

# Create summary stats from fitting results

In [ ]:
include("main_utils.jl")
include("data_setup.jl")
include("fit_utils.jl")
include("suppl_utils.jl")

In [ ]:
df_res = read_master_with_fit_summary(; update_data = false);

In [ ]:
df_dds = CSV.read("../dt_surveys_master/master_dds.csv", DataFrame);
df_obs = @pipe groupby(df_dds, [:key, :strat]) |>
                combine(_, :y => sum => :n_answer);
df_obs_all = @pipe @subset(df_obs, :strat .== "all") |> @select(_, Not(:strat))
df_EVI = read_EVI_summary(df_obs_all; filter_quantile = true);
clean_survey_key_names!(df_obs)
clean_survey_key_names!(df_EVI)

# WAIC weights

In [ ]:
include("main_utils.jl")
include("fit_utils.jl")
model_names = get_model_names()

In [ ]:
rem_lis = ["Danon 2013", "Leung 2017",  #"CoMix2",
    #"CoMix2_at", "CoMix2_be", "CoMix2_dk",
    #"CoMix2_ee", "CoMix2_gr", "CoMix2_hr",
    #"CoMix2_it", "CoMix2_pl", "CoMix2_pt",
]
df_res_ana = @subset(df_res, @byrow (:key in rem_lis) == false);
df_EVI_ana = @subset(df_EVI, @byrow (:key in rem_lis) == false);

In [ ]:
include("fit_utils.jl")
pl = plot_bar_waic_pretty(df_obs, df_res_ana, df_EVI_ana)
savefig(pl, "../fig/waic_comparison.png")
pl

### Sensitivity analysis of EVI

In [ ]:
include("fit_utils.jl")
df_EVI = read_EVI_summary(df_obs_all; filter_quantile = false);
pl = plot_EVI_sensitivity_qs(df_EVI)
savefig(pl, "../fig/EVI_sensitivity.png")
pl

## Fractional multinomial regression for meta-regression

In [ ]:
include("main_utils.jl")
include("fit_utils.jl")

In [ ]:
df_ana = prep_fmnl_vars(df_res_ana)
sort!(df_ana, :n_sample)
ytk = df_ana[:, :key_pri];
plot_bar_waic(df_obs, df_res_ana, ytk = ytk) #, df_EVI=df_EVI_nhm)

In [ ]:
include("main_utils.jl")
df_ana = prep_fmnl_vars(df_res_ana);
pred, Y, x_names = one_hot_encoding_multi_vars(df_ana);
@time chn = sample(model_fmnl(pred, Y), NUTS(), 2000; progress = true)
df_pred_cum = pred_fmnl_multi_vars(chn, pred)
plot_stacked_bar(df_pred_cum, model_names; xlabel="Prediced weight (%)")

In [ ]:
chn_res_multi = extract_chain_info(chn)
n_x = size(pred, 2)
forestplot_fmnl_multi_vars(chn_res_multi[Not([1, n_x+1]), :], x_names[2:end])

In [ ]:
chn_res = repeated_univariate_fmnl_reg(df_ana);

In [ ]:
include("fit_utils.jl")
pl = plot_meta_reg(chn_res, chn_res_multi)
plot!(pl, dpi=300)
savefig(pl, "../fig/meta_reg.png")
pl

## Fitting validation
TODO: will be removed. 

In [ ]:
include("main_utils.jl")
include("fit_utils.jl")

In [ ]:
model_names = get_model_names()

In [ ]:
key = "CoMix2"
plot_pdf_ccdf_validate(key)

In [ ]:
key = "Read_2014"
plot_pdf_ccdf_validate(key)

In [ ]:
include("main_utils.jl")
include("fit_utils.jl")
key = "Leung_2017_online"
plot_pdf_ccdf_validate(key)

In [ ]:
key = "Leung_2017_paper"
plot_pdf_ccdf_validate(key)

In [ ]:

pl = plot(; xlim = [0, 50], ylim = [-4, 0])
dd_home = res["dds"]["home"]
